In [1]:
import sys
sys.path.append('AttemptUsingImageGenerator/')

import intelliscope
from intelliscope import SelectDataset
from ml_utils import log_model_weights,log_epoch_data

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from sklearn.utils import shuffle
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.layers import Conv2D, MaxPool2D
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras import regularizers
print(tf.__version__)
import neptune
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

2.1.0


In [3]:
pwd

'/home/ubuntu/intelliscope/notebooks/physionet2016/pcg_ecg/Ramith/regular'

In [4]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNjBlODllY2ItMDcyZC00OGNmLThjZWYtZjM2MjE4Y2M5ZDM0In0="
os.environ['NEPTUNE_PROJECT']="icassp/Hybrid'"
#os.environ['NEPTUNE_NOTEBOOK_ID']="4897a223-8a9f-4f00-8cec-22eda690858d"
os.environ['NEPTUNE_NOTEBOOK_PATH']="notebooks_ramith/CNN_hybrid_transfer_learning.ipynb"

In [5]:
NEPTUNE_API_TOKEN="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNjBlODllY2ItMDcyZC00OGNmLThjZWYtZjM2MjE4Y2M5ZDM0In0="
PARAMS = {'ignore_blanks' : True,
          'epochs' : 15,
         'validation_split':0.2,
         'batch_size' : 20,
         'train_test_split':0.8,
         'dense_layers':80,
          'dropout':0.1,
         'dataset_path_ecg': '',
         'dataset_path_pcg': '',
         'image_type' : 'rgb',
         'im_type':6,
         'Read_Type':1,
          'opt':'adam',
          'data_balanced':False,
         }

PARAMS['dataset_path_pcg'],PARAMS['dataset_path_ecg'] = SelectDataset(0)

m = PARAMS['Read_Type']*3 + int(PARAMS['Read_Type']==0)
e = PARAMS['Read_Type']*6 + int(PARAMS['Read_Type']==0)*2


Loading... >  ICASSP_01
	=> /home/ubuntu/intelliscope/datasets/PhysioNet2016/preprocessed/training-a/original/ecg/01-cmor1.5-1.0/
	=> /home/ubuntu/intelliscope/datasets/PhysioNet2016/preprocessed/training-a/original/pcg/01-morl/


In [6]:
train_data_dir = [PARAMS['dataset_path_pcg'][:-1]+'_train/',PARAMS['dataset_path_ecg'][:-1]+'_train/']

In [7]:
k1,k2 = 272, 462
s_x = 82;
e_x = 544;
s_y = 43;
e_y = 315;

batch_size=PARAMS['batch_size']
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) # set validation split

def cropper(img):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    #print(img.shape)
    #plt.figure()
    #wplt.imshow(img[s_y:e_y,s_x:e_x])
    return img[s_y:e_y,s_x:e_x]
def generate_generator_multiple(generator,dir1, dir2, batch_size,subset):
    #k1, k2=360, 648
    k1, k2=272, 462
    genX1 = generator.flow_from_directory(dir1,
                                          target_size=(360, 648),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          subset=subset,
                                          shuffle=False, 
                                          seed=7)
    
    genX2 = generator.flow_from_directory(dir2,
                                          target_size=(360, 648),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          subset=subset,
                                          shuffle=False, 
                                         seed=7)
    while True:
            X1i, y1i = genX1.next()
            X2i, y2i = genX2.next()
            
            X1i_cropped = np.zeros((X1i.shape[0], k1, k2, 3))
            X2i_cropped = np.zeros((X2i.shape[0], k1, k2, 3))

            for i in range(X1i.shape[0]):      
                X1i_cropped[i] = cropper(X1i[i])
            for i in range(X2i.shape[0]):      
                X2i_cropped[i] = cropper(X2i[i])
                
            yield [X1i_cropped, X2i_cropped], y1i  #Yield both images and their mutual label
    
    
dual_training   = generate_generator_multiple(train_datagen,train_data_dir[0],train_data_dir[1],batch_size,'training')
dual_validation = generate_generator_multiple(train_datagen,train_data_dir[0],train_data_dir[1],batch_size,'validation')

pcg_training=train_datagen.flow_from_directory(train_data_dir[0],target_size=(k1,k2),class_mode = 'categorical', batch_size = batch_size, subset='training', shuffle=False,  seed=7)
pcg_validation=train_datagen.flow_from_directory(train_data_dir[0],target_size=(k1,k2),class_mode = 'categorical', batch_size = batch_size, subset='validation', shuffle=False,  seed=7)

ecg_training=train_datagen.flow_from_directory(train_data_dir[1],target_size=(k1,k2),class_mode = 'categorical', batch_size = batch_size, subset='training', shuffle=False,  seed=7)
ecg_validation=train_datagen.flow_from_directory(train_data_dir[1],target_size=(k1,k2),class_mode = 'categorical', batch_size = batch_size, subset='validation', shuffle=False,  seed=7)

neptune.init('icassp/Hybrid',NEPTUNE_API_TOKEN)
exp = neptune.create_experiment(name='CNN_hybrid_udith_test',params=PARAMS,upload_source_files=['exp.ipynb'],tags=['HS','ECG','ICASSP', 'PCG_only'],upload_stdout=True)


Found 2254 images belonging to 2 classes.
Found 562 images belonging to 2 classes.


/home/ubuntu/core-env/lib/python3.6/site-packages/keras_preprocessing/image/utils.py:179: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn('Using ".tiff" files with multiple bands '


Found 2254 images belonging to 2 classes.
Found 562 images belonging to 2 classes.


NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/icassp/Hybrid/e/HYBRID-91


In [8]:
model_pcg = tf.keras.models.load_model('/home/ubuntu/intelliscope/notebooks/physionet2016/pcg/models/Experiment(PCG-50).h5')
model_ecg = tf.keras.models.load_model('/home/ubuntu/intelliscope/notebooks/physionet2017/ecg/models/Experiment(ECG-24).h5')

print('PCG')
#model_pcg.summary()
print('ECG')
#model_ecg.summary()

# warning due to Instance Norm

PCG
ECG


In [9]:
pcg_layer='flatten_1'
ecg_layer='flatten'

HS_MODEL= tf.keras.Model(inputs=model_pcg.input, outputs=model_pcg.get_layer(pcg_layer).output)
ECG_MODEL= tf.keras.Model(inputs=model_ecg.input, outputs=model_ecg.get_layer(ecg_layer).output)
HS_MODEL.trainable=True
ECG_MODEL.trainable=True

hs_input = tf.keras.layers.Input(shape=(272,462,m))
ecg_input = tf.keras.layers.Input(shape=(272,462,m))

hs_model = HS_MODEL(hs_input)
ecg_model = ECG_MODEL(ecg_input)

print(ecg_model.get_shape())
print(hs_model.get_shape())

hs_model_ = tf.keras.layers.LayerNormalization()(hs_model)
ecg_model_ = tf.keras.layers.LayerNormalization()(ecg_model)
concat     = tf.keras.layers.concatenate([hs_model_, ecg_model_])
dropout    = tf.keras.layers.Dropout(PARAMS['dropout'])(concat)
dense      = tf.keras.layers.Dense(PARAMS['dense_layers'], activation='relu')(dropout)
dense_1    = tf.keras.layers.Dense(20, activation='relu')(dense)
output = tf.keras.layers.Dense(2, activation='softmax')(dense_1)

model_hybrid = tf.keras.Model(inputs=[hs_input, ecg_input], outputs=[output])

(None, 3840)
(None, 3840)


In [10]:
def reset_weights(model):
    session = K.get_session()
    for layer in model.layers:
        if isinstance(layer, Dense):
            layer.W.initializer.run(session=session)
            layer.b.initializer.run(session=session)
        if isinstance(layer, Conv2D):
            layer.W.initializer.run(session=session)
            layer.b.initializer.run(session=session)
        else:
            print(layer, "not reinitialized")

In [11]:
model=model_pcg


In [12]:

model.compile(optimizer=keras.optimizers.Adam(),  # Optimizer
              # Loss function to minimize
              loss=keras.losses.CategoricalCrossentropy(),
              # List of metrics to monitor
              metrics=[keras.metrics.CategoricalAccuracy()])
model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))



if model==model_pcg:
    print('pcg only evaluation')
    model.fit_generator(
        pcg_training,
        steps_per_epoch = 1803 // batch_size,
        validation_data = pcg_validation,
        validation_steps = 451 // batch_size,
        epochs =PARAMS['epochs'],
        callbacks=[
                    tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: log_epoch_data(logs, exp)),]
    )
    
if model==model_ecg:
    print('pcg only evaluation')
    model.fit_generator(
        ecg_training,
        steps_per_epoch = 1803 // batch_size,
        validation_data = ecg_validation,
        validation_steps = 451 // batch_size,
        epochs =PARAMS['epochs'],
        callbacks=[
                    tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: log_epoch_data(logs, exp)),]
    )
    
if model==model_hybrid:
    print('pcg only evaluation')
    model.fit_generator(
        dual_training,
        steps_per_epoch = 1803 // batch_size,
        validation_data = dual_validation,
        validation_steps = 451 // batch_size,
        epochs =PARAMS['epochs'],
        callbacks=[
                    tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: log_epoch_data(logs, exp)),]
    )

pcg only evaluation
Instructions for updating:
Please use Model.fit, which supports generators.


Instructions for updating:
Please use Model.fit, which supports generators.


  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 90 steps, validate for 22 steps
Epoch 1/15
89/90 [============================>.] - ETA: 1s - loss: 0.7062 - categorical_accuracy: 0.7449

KeyError: 'sparse_categorical_accuracy'

In [ ]:
fig = plt.figure()
plt.plot(history.history['sparse_categorical_accuracy'], label='sparse_categorical_accuracy')
plt.plot(history.history['val_sparse_categorical_accuracy'], label = 'val_sparse_categorical_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
test_loss, test_acc = model.evaluate([x_test1,resized_x_test2],  y_test, verbose=2)

fig.canvas.draw()
curve = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
curve = curve.reshape(fig.canvas.get_width_height()[::-1] + (3,))

neptune.log_image('learning_curve', curve)
neptune.log_metric('Test Accuracy',test_acc)
neptune.log_metric('Test Loss',test_loss)
print(test_acc)

In [ ]:

loss, acc = model.evaluate([x_test1,resized_x_test2],  y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

In [36]:
keras_model_path = "/home/ec2-user/SageMaker/notebooks_ramith/out/"  + str(exp) + '.h5'  

model.save(keras_model_path)  # save() should be called out of strategy scope

neptune.log_artifact(keras_model_path)


In [37]:
from sklearn.metrics import classification_report

y_pred = model.predict([x_test1,resized_x_test2], batch_size=100, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

z = classification_report(y_test, y_pred_bool)
print(z)

neptune.log_text('classification_report', z)

632/632 [==============================] - 6s 9ms/sample
              precision    recall  f1-score   support

           0       0.42      0.15      0.23       188
           1       0.72      0.91      0.80       444

    accuracy                           0.69       632
   macro avg       0.57      0.53      0.51       632
weighted avg       0.63      0.69      0.63       632



In [40]:
#!pip install neptune-contrib

In [45]:
!pip install scikit-plot -q

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [46]:
from neptunecontrib.monitoring.metrics import *

threshold = 0.5

log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
log_classification_report(y_test, y_pred[:, 1] > threshold)
log_class_metrics(y_test, y_pred[:, 1] > threshold)
log_class_metrics_by_threshold(y_test, y_pred[:, 1])
log_roc_auc(y_test, y_pred)
log_precision_recall_auc(y_test, y_pred)
log_brier_loss(y_test, y_pred[:, 1])
log_log_loss(y_test, y_pred)
log_ks_statistic(y_test, y_pred)
log_cumulative_gain(y_test, y_pred)
log_lift_curve(y_test, y_pred)
log_prediction_distribution(y_test, y_pred[:, 1])

In [21]:

neptune.stop()

#### 